In [58]:
import pandas as pd
import numpy as np
import datetime as dt
import itertools
from IPython.core.display import display, HTML
import sklearn


display(HTML("<style>.container { width : 98% !important; }</style>"))

pd.set_option('display.max_columns',500)

In [59]:
data = pd.read_csv('/Users/APinkerton/Downloads/data.csv',encoding = 'unicode_escape')

In [67]:
# Cleaning up column headers, ranking customer ID's starting from 1 as unique identifiers
cols1 = data.columns 
cols2 = [] 
for x in cols1:
    cols2.append(x.lower().strip().replace(' ','_'))
data.columns = cols2
data = data[~(data['customerid'].isnull()) | (data['customerid']==np.inf)].copy()
data['customerid'] = data['customerid'].astype(int)
data#[['invoiceno','invoicedate']]
data['customerid'].rank(method='dense',ascending=True).astype(int)
len(np.unique(data['invoiceno']))

22190

In [78]:
# Starting to group by the customer id, collapsing transaction items into one DataFrame cell list
# Also merging on customer rank and then ranking the transactions of the customers to put them in chronological order
grp = data.groupby(['invoiceno','customerid'])['stockcode'].agg(['unique']).reset_index()
df = pd.merge(grp,data[['invoiceno','invoicedate']],left_on='invoiceno',right_on='invoiceno',how='left')


custos = pd.DataFrame(df['customerid'].rank(ascending=True,method='dense').astype(int).reset_index())
custos.columns = ['index','custo_rank']

rnks = pd.DataFrame(df.groupby(['customerid'])[['invoiceno']].rank(ascending=True,method='dense').astype(int).reset_index())
rnks.columns = ['index','xaction_rank']
df = pd.merge(df,custos,left_index=True,right_on='index')
df = pd.merge(df,rnks,left_index=True,right_on='index')
df.drop_duplicates(['invoiceno'],inplace=True)
# Getting item counts for each transaction
sz = []
for a in df['unique']:
    sz.append(len(a))

In [87]:
df['itm_ct'] = sz
res_df = df[['custo_rank','xaction_rank', 'itm_ct', 'unique']].copy()
res_df.sort_values(by=['custo_rank','xaction_rank'],ascending=True,inplace=True)
empt=[]
for lst in df['unique']:
    itms = []
    for itm in lst:
        itms.append(str(itm))
    empt.append(itms)
res_df['unique'] = empt
res_df

,custo_rank,xaction_rank,itm_ct,unique
37126,1,1,1,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
398918,1,2,1,"[22633, 22632]"
10516,2,1,31,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
43443,2,2,29,"[22960, 22913, 22912, 22914]"
99433,2,3,24,[21756]
...,...,...,...,...
379544,4371,15,39,[23843]
390856,4371,16,50,"[22178, 23144]"
137312,4372,1,27,[M]
284483,4372,2,38,[21258]


In [88]:
res_df.to_csv('/Users/APinkerton/Downloads/xactions.csv')